# Análise de Venda de Medicamentos Controlados e Antimicrobianos - Medicamentos Industrializados

Projeto de análise de vendas de medicamentos controlados e antimicrobianos exclusivamente industrializados na Região Metropolitana da Baixada Santista (RMBS) composta por nove municípios no litoral do estado de São Paulo, através de dados extraídos do Sistema Nacional de Gerenciamento de Produtos Controlados (SNGPC) e disponibilizados no [portal de dados abertos](https://dados.gov.br/dados/conjuntos-dados/venda-de-medicamentos-controlados-e-antimicrobianos---medicamentos-industrializados) da Agência Nacional de Vigilância Sanitária (Anvisa). Através da análise das informações de vendas, dados geográricos, perfil de pacientes e características de medicamentos, o objetivo é extrair insights de negócios, sugerir soluções aos problemas identificados e apresentar propostas de aprimoramento.

## Demanda do negócio

- Compreender as tendências, padrões e características das vendas farmacêuticas.
- Construção de perfis de pacientes com base nos mendicamentos receitados.
- Compreender a demanda de medicamentos com base no perfil dos pacientes, por tempo e por município.
- Obter insights, identificar oportunidades e propor soluções a problemas.
- Elaboração de painel de informações de medicamentos baseado em filtros.

## Compreensão dos dados

Os dados que serão utilizados na análise compreendem o período de uma ano, outubro de 2020 até setembro de 2021, que integram um conjunto de doze arquivos em formato "CSV". Os dados foram extraídos do Sistema Nacional de Gerenciamento de Produtos Controlados (SNGPC), provenientes apenas de farmácias e drogarias privadas que periodicamente devem enviar os dados a respeito de todas as vendas realizadas de medicamentos sujeitos à escrituração no SNGPC. Os dados foram disponibilizados no [portal de dados abertos](https://dados.gov.br/dados/conjuntos-dados/venda-de-medicamentos-controlados-e-antimicrobianos---medicamentos-industrializados) da Agência Nacional de Vigilância Sanitária (Anvisa).


## Tópico da análise

- Construir uma ABT (analytical base table).

## Dicionário de dados

**ANO_VENDA:** Ano da venda do medicamento.<br>
**MES_VENDA:** Mês da venda do medicamento.<br>
**UF_VENDA:** Unidade Federativa do endereço da farmácia ou drogaria, cadastrado no banco de dados da Anvisa, representando a UF onde ocorreu a venda.<br>
**MUNICIPIO_VENDA:** Município do endereço da farmácia ou drogaria, cadastrado no banco de dados da Anvisa, representando o Município onde ocorreu a venda.<br>
**PRINCIPIO_ATIVO:** Nome do princípio ativo do medicamento industrializado, conforme cadastrado no registro do medicamento, no banco de dados da Anvisa.<br>Quando um medicamento tem mais de um princípio ativo, cada um deles é separado pelo caractere “+”.<br>Ex.: “PRINCÍPIO ATIVO 1 + PRINCÍPIO ATIVO 2”<br>
**DESCRICAO_APRESENTACAO:** Uma Apresentação de Medicamento representa O modo como um medicamento é apresentado na embalagem. Exemplo: Medicamento X, pode ter duas apresentações diferentes:<br>• Apresentação 1:<br>Uma caixa com 1 blister de alumínio com 20 comprimidos, cada comprimido com 5 mg de princípio ativo.<br>Nesse caso, a descrição da apresentação seria:<br>“5 MG COM CT BL AL X 20”<br>• Apresentação 2:<br>Uma caixa com 1 frasco de vidro com 50 mL de um xarope, com concentração do princípio ativo de 15 mg por mL.<br>Nesse caso, a descrição da apresentação seria:<br>15MG/ML XPE CT FR VD x 50 ML<br>Esses exemplos representam descrições de apresentações diferentes para um mesmo medicamento.<br>Os termos utilizados na descrição das apresentações seguem o disposto no Vocabulário Controlado da Anvisa, disponível no link:<br>[http://portal.anvisa.gov.br/documents/33836/2501339/Vocabul%C3%A1rio+Controlado/fd8fdf08-45dc-402a-8dcf-fbb3fd21ca75](http://portal.anvisa.gov.br/documents/33836/2501339/Vocabul%C3%A1rio+Controlado/fd8fdf08-45dc-402a-8dcf-fbb3fd21ca75)<br>
**QTD_VENDIDA:** Quantidade vendida de caixas ou frascos do medicamento.<br>
**UNIDADE_MEDIDA:** Indica se a quantidade vendida do medicamento foi de caixas ou frascos.<br>
**CONSELHO_PRESCRITOR:** Conselho de Classe do profissional que prescreveu o medicamento vendido.<br>
**UF_CONSELHO_PRESCRITOR:** Unidade Federativa do Conselho de Classe do profissional que prescreveu o medicamento vendido.<br>
**TIPO_RECEITUARIO:** Tipo de receituário utilizado na prescrição.<br>Valores e respectivos tipos de receituário:<br>1 – Receita de Controle Especial em 2 vias (Receita Branca);<br>2 – Notificação de Receita B (Notificação Azul);<br>3 – Notificação de Receita Especial (Notificação Branca);<br>4 – Notificação de Receita A (Notificação Amarela);<br>5 – Receita Antimicrobiano em 2 vias.<br>
**CID10:** Classificação Internacional de Doença (aplicável apenas a medicamentos antimicrobianos).<br>
**SEXO:** Sexo do paciente (aplicável apenas a medicamentos antimicrobianos).<br>Valor 1 para o sexo masculino, valor 2 para o sexo feminino.<br>
**IDADE:** Valor numérico que representa a idade do paciente, em meses ou anos (aplicável apenas a medicamentos antimicrobianos).<br>
**UNIDADE_IDADE:** Unidade de medida da idade do paciente, que pode ser em meses ou anos (aplicável apenas a medicamentos antimicrobianos).<br>Valor 1 para unidade de medida em anos, valor 2 para unidade de medida em meses.<br>

-------------------------------------

### Importação de bibliotecas

In [1]:
from pyspark.sql import SparkSession
import glob


### Criação e iniciação de uma sessão Spark

In [2]:
appName = 'PySpark - ABT de Vendas Farmaceuticas'

spark = SparkSession.builder \
    .appName(appName) \
    .config('spark.driver.memory', '8g') \
    .config('spark.driver.cores', '2') \
    .config('spark.executor.memory', '8g') \
    .config('spark.executor.cores', '4') \
    .master("local[*]") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

spark


### Criação do dataset a partir da leitura dos arquivos *.csv

In [3]:
# Lista de caminhos para os arquivos CSV
caminhos_csv = glob.glob('dados/*.csv')

# DataFrame vazio
df = None

# Loop para ler e unir os arquivos CSV
for caminho_csv in caminhos_csv:
    df_temp = spark.read.csv(caminho_csv, sep=';', header=True, encoding='cp1252', inferSchema=True)
    
    if df is None:
        df = df_temp
    else:
        df = df.union(df_temp)

# Exibir o esquema do DataFrame combinado
df.printSchema()

In [ ]:
# Exibir as 5 primeiras linhas do DataFrame combinado
df.show(15, truncate=False)


+---------+---------+--------+---------------+------------------------+---------------------------------------------------------+-----------+--------------+-------------------+----------------------+----------------+-----+----+-----+-------------+
|ANO_VENDA|MES_VENDA|UF_VENDA|MUNICIPIO_VENDA|PRINCIPIO_ATIVO         |DESCRICAO_APRESENTACAO                                   |QTD_VENDIDA|UNIDADE_MEDIDA|CONSELHO_PRESCRITOR|UF_CONSELHO_PRESCRITOR|TIPO_RECEITUARIO|CID10|SEXO|IDADE|UNIDADE_IDADE|
+---------+---------+--------+---------------+------------------------+---------------------------------------------------------+-----------+--------------+-------------------+----------------------+----------------+-----+----+-----+-------------+
|2020     |10       |BA      |JEQUIÉ         |BROMAZEPAM              |6 MG CAP GEL C/ MCGRAN AP CT BL AL PVC ACLAR PL X 30     |3          |FRASCO        |CRM                |BA                    |2               |NULL |NULL|NULL |NULL         |
|2020   

In [ ]:
# Contar o número de linhas no DataFrame
qtde_linhas = df.count()
print(f'\nO DataFrame tem {qtde_linhas} linhas.')

# Obter o número de colunas no DataFrame
qtde_colunas = len(df.columns)
print(f'\nO DataFrame tem {qtde_colunas} colunas.')


O DataFrame tem 73950048 linhas.

O DataFrame tem 15 colunas.


### Análise dos dados para construção da ABT

In [6]:
# Criando uma View temporária para uso do SparkSQL
df.createOrReplaceTempView('tb_medicamentos')

In [7]:
# Verificando a volume total de registros dos 9 municípios da RMBS
qtde_linhas_rmbs = spark.sql('''
    SELECT
        COUNT(*) AS QTDE_TOTAL
    FROM 
        tb_medicamentos
    WHERE
        MUNICIPIO_VENDA IN ('BERTIOGA', 'CUBATÃO', 'GUARUJÁ', 'ITANHAÉM', 'MONGAGUÁ', 'PERUÍBE', 'PRAIA GRANDE', 'SANTOS', 'SÃO VICENTE')
''')

qtde_linhas_rmbs.show()


+----------+
|QTDE_TOTAL|
+----------+
|    748950|
+----------+



In [8]:
# Coletar o resultado da primeira consulta
qtde_rmbs = qtde_linhas_rmbs.first()['QTDE_TOTAL']

# Verificando a volume total de registros de cada 1 dos 9 municípios da RMBS
spark.sql('''
    SELECT
        MUNICIPIO_VENDA,
        COUNT(*) AS QTDE_TOTAL,
        ROUND(100*(COUNT(*)/{}), 3) AS QTDE_TOTAL_PERCENT
    FROM 
        tb_medicamentos
    WHERE
        MUNICIPIO_VENDA IN ('BERTIOGA', 'CUBATÃO', 'GUARUJÁ', 'ITANHAÉM', 'MONGAGUÁ', 'PERUÍBE', 'PRAIA GRANDE', 'SANTOS', 'SÃO VICENTE')
    GROUP BY
        MUNICIPIO_VENDA
    ORDER BY 
        QTDE_TOTAL DESC
'''.format(qtde_rmbs)).show()


+---------------+----------+------------------+
|MUNICIPIO_VENDA|QTDE_TOTAL|QTDE_TOTAL_PERCENT|
+---------------+----------+------------------+
|         SANTOS|    214327|            28.617|
|   PRAIA GRANDE|    135360|            18.073|
|    SÃO VICENTE|    108158|            14.441|
|        GUARUJÁ|     94778|            12.655|
|        PERUÍBE|     44902|             5.995|
|       ITANHAÉM|     42737|             5.706|
|        CUBATÃO|     41304|             5.515|
|       BERTIOGA|     34645|             4.626|
|       MONGAGUÁ|     32739|             4.371|
+---------------+----------+------------------+



In [9]:
# Selecionando os dados que irão compor a ABT
abt_rmbs = spark.sql('''
    SELECT
        ANO_VENDA,
        MES_VENDA,
        UF_VENDA,
        MUNICIPIO_VENDA,
        PRINCIPIO_ATIVO,
        DESCRICAO_APRESENTACAO,
        QTD_VENDIDA,
        UNIDADE_MEDIDA,
        CONSELHO_PRESCRITOR,
        UF_CONSELHO_PRESCRITOR,
        TIPO_RECEITUARIO,
        CID10,
        SEXO,
        IDADE,
        UNIDADE_IDADE,
        TO_DATE(concat(ANO_VENDA, lpad(MES_VENDA, 2, '0'), '01'), 'yyyyMMdd') AS DATA_REF,
        CURRENT_DATE AS DATA_PROC 
    FROM
        tb_medicamentos
    WHERE
        MUNICIPIO_VENDA IN ('BERTIOGA', 'CUBATÃO', 'GUARUJÁ', 'ITANHAÉM', 'MONGAGUÁ', 'PERUÍBE', 'PRAIA GRANDE', 'SANTOS', 'SÃO VICENTE')    
''')

abt_rmbs.show(truncate=False)

+---------+---------+--------+---------------+--------------------------------------------------------+-----------------------------------------------------+-----------+--------------+-------------------+----------------------+----------------+-----+----+-----+-------------+----------+----------+
|ANO_VENDA|MES_VENDA|UF_VENDA|MUNICIPIO_VENDA|PRINCIPIO_ATIVO                                         |DESCRICAO_APRESENTACAO                               |QTD_VENDIDA|UNIDADE_MEDIDA|CONSELHO_PRESCRITOR|UF_CONSELHO_PRESCRITOR|TIPO_RECEITUARIO|CID10|SEXO|IDADE|UNIDADE_IDADE|DATA_REF  |DATA_PROC |
+---------+---------+--------+---------------+--------------------------------------------------------+-----------------------------------------------------+-----------+--------------+-------------------+----------------------+----------------+-----+----+-----+-------------+----------+----------+
|2020     |10       |SP      |MONGAGUÁ       |ACETATO DE PREDNISOLONA + GATIFLOXACINO SESQUI-HIDRATADO|3 M

In [10]:
# Exibir o esquema da ABT
abt_rmbs.printSchema()

root
 |-- ANO_VENDA: integer (nullable = true)
 |-- MES_VENDA: integer (nullable = true)
 |-- UF_VENDA: string (nullable = true)
 |-- MUNICIPIO_VENDA: string (nullable = true)
 |-- PRINCIPIO_ATIVO: string (nullable = true)
 |-- DESCRICAO_APRESENTACAO: string (nullable = true)
 |-- QTD_VENDIDA: integer (nullable = true)
 |-- UNIDADE_MEDIDA: string (nullable = true)
 |-- CONSELHO_PRESCRITOR: string (nullable = true)
 |-- UF_CONSELHO_PRESCRITOR: string (nullable = true)
 |-- TIPO_RECEITUARIO: decimal(1,0) (nullable = true)
 |-- CID10: string (nullable = true)
 |-- SEXO: integer (nullable = true)
 |-- IDADE: double (nullable = true)
 |-- UNIDADE_IDADE: integer (nullable = true)
 |-- DATA_REF: date (nullable = true)
 |-- DATA_PROC: date (nullable = false)



### Salvando a ABT em formato parquet

In [11]:
# Convertendo a ABT (sql) para o formato 'parquet'
caminho = 'dados/ABT/'
abt_rmbs.write.partitionBy('DATA_REF').parquet(caminho, mode='overwrite')

In [12]:
# Validando a quantidade de linhas
read_abt_rmbs = spark.read.format('parquet').load(caminho)

print(f'\nA ABT \'parquet\' tem {read_abt_rmbs.count()} linhas.')
print(f'\nA ABT \'sql\' tem {qtde_rmbs} linhas.')


A ABT 'parquet' tem 748950 linhas.

A ABT 'sql' tem 748950 linhas.
